In [ ]:
'''
Feed-forward neural network with 5 hidden dense layers to predict output distribution.
Preprocessing code written by Prof. Horvat and edited by Meal Swipes for Late Days, network code written by Meal Swipes for Late Days.
Input is a (batch_size x 26) tensor representing the input wave spectrum, 
output is a (batch size x 49) tensor representing the output floe size distribution.
'''
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat,savemat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive 

In [ ]:
# Define file locations
include_last = True;

drive.mount('/content/drive', force_remount=True)
# locstr = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/FSTD-Ongoing-Work/Neural-Net-Waves/'
locstr = '/content/drive/Shared drives/DeepWIFF/'

training_string = locstr + 'Data/training_data_errtol'

if not include_last:
    training_string = training_string + '_nolast'

training_string = training_string + '.mat'


Mounted at /content/drive


In [ ]:
'''
Loads training data and performs preprocessing to remove degenerate inputs.
There is no need to run this cell once the pickle files have been saved once.
'''
training_data = loadmat(training_string)
misc_data = loadmat(locstr + 'Data/NN_params.mat')

input_vec = training_data['input_vec'].T
output_vec = training_data['output_vec'].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T
Lambda = misc_data['lambda']


num_frac = np.sum(output_vec,axis=1)
# classifier = load_model('Classifiers/KERAS-Classifier-25x25.h5')
# thresh = 0.11;
# y_pred_class = classifier.predict(input_vec)
# usable = y_pred_class[:,1] > thresh

usable = num_frac > 0
print('Out of ' + str(len(num_frac)) + ', ' + str(sum(usable)) + ' can be used')

ice_thick = input_vec[:,-1]
wave_spec = input_vec[:,:25]

# wave_spec[wave_spec < 10**-8] = 0
wave_energy = np.sum(wave_spec*np.expand_dims(dFreq,axis=1).T,axis=1)
peak_loc = np.argmax(wave_spec,axis=1)
peak_freq = Freq[peak_loc]

NN_input = input_vec[usable]
NN_output = output_vec[usable]



Out of 1830900, 404503 can be used


In [ ]:
'''
Loads training and testing data from pickle file.
'''
import pickle

try:

    X_train = pickle.load( open(locstr +  "Data/" + "X_train.p", "rb" ) )
    X_test = pickle.load( open( locstr + "Data/" + "X_test.p", "rb" ) )
    y_train = pickle.load( open(locstr + "Data/" + "y_train.p","rb"))
    y_test = pickle.load( open(locstr + "Data/" + "y_test.p","rb"))
    
except:
        
        print('exception in loading segmented data')
        X_train, X_test, y_train, y_test = train_test_split(NN_input,NN_output,test_size=0.3)
        
        pickle.dump(X_train, open( locstr + "Data/" + "X_train.p", "wb" ) )
        pickle.dump(X_test, open( locstr + "Data/" + "X_test.p", "wb" ) )
        pickle.dump(y_train, open( locstr + "Data/" + "y_train.p", "wb" ) )
        pickle.dump(y_test, open( locstr + "Data/" + "y_test.p", "wb" ) )

In [ ]:
#Feed-forward model definition.
class FeedForwardModel(tf.keras.Model):
    def __init__(self,num_bins):
        """
		Intializer for a Feed Forward Neural Network model. Intializes all the trainable variables and hyperparameters to be used 
        throughout the model.

		:param num_bins:  integer tensor, denoting the 1D dimension of the output. Will be a histogram with num_bins bins, and a 
                            probability distribution.
		"""
        super(FeedForwardModel, self).__init__()
        
        # Training hyperparamaters
        self.batch_size = 256
        self.learning_rate = .001
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = self.learning_rate)
        self.num_bins = num_bins

        # Dense layers hyperparameters
        self.hidden_architecture = [100, 100, 100, 100, 100]
        self.activation = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.regularizer = {} #{'kernel_regularizer': keras.regularizers.l2(0.001), 'bias_regularizer': keras.regularizers.l2(0.001)}

        #Layers
        self.hidden_layers = [tf.keras.layers.Dense(size, activation = self.activation, kernel_initializer='random_normal', **self.regularizer) for size in self.hidden_architecture]
        self.final_layer = tf.keras.layers.Dense(num_bins, activation = "softmax", kernel_initializer='random_normal')

    def call(self,inputs):
        """
		Call function for Feed Forward Neural Network model. Passes input through MixtureDensity network and returns tensor of 
        probability distributions. 
		
        :param input:  tensor, tensor representing multiple 1d vectors to be passed through the call function 

        :return distribution: a tensor of discrete/continuous probability distributions
		"""

        hidden_layer_output = self.hidden_layers[0](inputs)
        for i in range(1,len(self.hidden_layers)):
            hidden_layer_output = self.hidden_layers[i](hidden_layer_output)
        final_layer_output = self.final_layer(hidden_layer_output)
        return final_layer_output

In [ ]:
#Creating the Feed Forward model
FFNN = FeedForwardModel(49)

"""
Compiles all the data for training and testing the Mixture Desnity model. Creates a list of different loss functions to keep track of.
"""
losses = ["mse", "kld", "mae"]
FFNN.compile(optimizer = FFNN.optimizer, loss = losses[0], metrics = ["kullback_leibler_divergence","mean_squared_error", "mean_absolute_error"])

# Output name
outstr = locstr + 'Network Weights/FFNN-' + str(FFNN.num_bins)
if not include_last:
    outstr = outstr + '-nolast'        

#Adds early stopping, if so desired
es = EarlyStopping(monitor='val_loss', verbose=1, patience=10);

# Reduce learning rate on plateau
lradj = keras.callbacks.ReduceLROnPlateau()

#Initializing necessary TensorBoard metrics for better metrics
tb_log = tf.keras.callbacks.TensorBoard(
    log_dir = locstr + "logs",
    histogram_freq = 5,
    write_graph = True,
    update_freq= 50,
    profile_batch=2,
    embeddings_freq=1)

"""
Trains and tests the mixture density network. Validation_data = data to be tested. Automatically trains and tests for 100 epochs
"""
history = FFNN.fit(X_train,y_train, validation_data = (X_test,y_test), verbose = 1, batch_size = FFNN.batch_size, epochs=100, callbacks = [lradj])

#Saving the weights of the feed forward model
FFNN.save(outstr) #save weights to a .h5 file

Epoch 1/100
1107/1107 [==============================] - 6s 6ms/step - loss: 0.0012 - kullback_leibler_divergence: 0.5711 - mean_squared_error: 0.0012 - mean_absolute_error: 0.0116 - val_loss: 8.0068e-04 - val_kullback_leibler_divergence: 0.3749 - val_mean_squared_error: 8.0068e-04 - val_mean_absolute_error: 0.0093
Epoch 2/100
1107/1107 [==============================] - 6s 5ms/step - loss: 7.1774e-04 - kullback_leibler_divergence: 0.3357 - mean_squared_error: 7.1774e-04 - mean_absolute_error: 0.0082 - val_loss: 6.1984e-04 - val_kullback_leibler_divergence: 0.2846 - val_mean_squared_error: 6.1984e-04 - val_mean_absolute_error: 0.0075
Epoch 3/100
1107/1107 [==============================] - 6s 6ms/step - loss: 6.0690e-04 - kullback_leibler_divergence: 0.2751 - mean_squared_error: 6.0690e-04 - mean_absolute_error: 0.0071 - val_loss: 7.4738e-04 - val_kullback_leibler_divergence: 0.3778 - val_mean_squared_error: 7.4738e-04 - val_mean_absolute_error: 0.0069
Epoch 4/100
1107/1107 [==========

NotImplementedError: ignored

In [ ]:
#For assessing network performance, load a saved model.
#model_name = 'Full-100x100x100x100x100-kld-FINAL_2.h5'
NN_fullnet = tf.saved_model.load(locstr + 'Network Weights/FFNN-' + str(FFNN.num_bins))

In [ ]:
# q = random.randint(0,y_test.shape[0])
# plt.semilogx(Rcent,y_test[q,:])

# test_out = NN_fullnet.predict(np.expand_dims(X_test[q,:],axis=1).T).T

# plt.semilogx(Rcent,test_out)

# print('MAE value here is ' + str(sum(abs(np.subtract(np.expand_dims(y_test[q,:],axis=1),test_out)))))
training_data = loadmat(training_string)
misc_data = loadmat(locstr + 'Data/NN_params.mat')

input_vec = training_data['input_vec'].T
output_vec = training_data['output_vec'].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T
Lambda = misc_data['lambda']
test_out = NN_fullnet.predict(X_test)

In [ ]:
errmat = np.sum(np.abs((test_out - y_test)),axis=1)
errmat2 = np.mean((test_out - y_test)**2,axis=1)
testE = np.sum(X_test[:,:25]*dFreq.T,axis=1)
testH = X_test[:,-1]

print(errmat.shape)
print(testE.shape)
print(testH.shape)

print('SAE is: ' + str(np.nanmean(errmat)))
print(">10% SAE is: " + str(np.sum(errmat > 0.1)/len(X_test)))
print('MSE is: ' + str(np.nanmean(errmat2)))


(121351,)
(121351,)
(121351,)
SAE is: 0.13769825268025623
>10% SAE is: 0.2937759062554079
MSE is: 0.00022182967692278992
